# 03 – Preprocessing & Feature Engineering
### Car Price Prediction Using Machine Learning
Group Assignment 02 - CCS3012 - Data Analytics  
Submission Date: 16th September 2025

---

### **Group 11**
-  **FC211034 - N.D. Samararathne Kodikara**
-  **FC211013 - N.W.V. Tharindu Pabasara**
-  **FC211025 - W.M.M.C.B. Wijesundara**



---

### **Supervisor**
**Ms. Dilmi Praveena**  
*Faculty of Computing*  
*University of Sri Jayewardenepura*

---



## 📌 Objective
Prepare the cleaned car price dataset for modeling by performing feature engineering, encoding categorical variables, scaling numerical features, and creating train/test splits.
**Key Tasks:**
- Load the cleaned dataset from `data/processed/car_price_cleaned.csv`.
- Engineer new features
- Encode categorical variables
- Scale or normalize numerical features (e.g., `price`, `mileage`, `levy`, `engine_volume`).
- Explore feature-target relationships with visualizations.
- Split dataset into train/test sets.
- Save the preprocessed dataset.

---

### 📂 Input  
- `clean_data.csv` saved in `Data/processed/`  


### 📦 Output  


---

### 📊 Dataset Overview
**Dataset:** Car price dataset cleaned.  
**columns include**:   
- `id`, `price` (target variable),  `levy`,  `manufacturer`,  `model`,  `prod_year`,  `category`,  `leather_interior`,  `fuel_type`, `engine_volume`,  `mileage`,  `cylinders`,  `gear_box_type`,  `drive_wheels`,  `doors`,  `wheel`, `color`,  `airbags`,  `turbo` (derived in Notebook 01)


> **Dataset stats:** 19,237 rows × 19 features | Target variable: `price` | Problem type: Regression


## Setup & imports